# Spike-Timing Dependent Plasticity (STDP)
## STDP
Now we shall implement some long-term plasticity. Spike-Timing dependent Plasticity (STDP) is a type of neuronal activity-dependent synaptic plasticity, in which the strength of a synapse increases or decreases depending on the time difference between a spike from the presynaptic and a spike from the postsynaptic neuron (from now on we refer to these spikes as pre-spikes and post-spikes, respectively).
While different subtypes of STDP can be found in the literature \cite{Abbott2000}, the most widely observed and discussed variety is ``Hebbian`` or asymmetric STDP, which entails long-term potentiation (LTP) after a pre-post spike pair, and long-term depression (LTD) of the synaptic weight after a post-pre spike pair. Excitatory synapses in the hippocampus and cortex are subject to LTP and LTD via Hebbian STDP \cite{Bi1998, Markram1997}.

The dependence on the difference in spike timing can be modeled by an exponentially decaying shape as a function of the time difference between a presynaptic and a postsynaptic spike $ \Delta t = t_{\rm prespike} - t_{\rm postspike}$ (Fig.~\ref{fig:STDP-window}). The change in the excitatory synaptic strength (weight) $w_\mathrm{e}$ at the moment of a pre- or postsynaptic spike obeys:

$$
\begin{align}
 \Delta w_\mathrm{e} & = \begin{cases} 
   A_{\mathrm{LTP}} \exp ({\frac{\Delta t}{\tau_{\mathrm{LTP}}})} &  \forall \Delta t<0 \\
   A_{\mathrm{LTD}} \exp ({ - \frac{\Delta t}{\tau_{\mathrm{LTD}}})}  &   \forall \Delta t>0 \\
   0 &   \forall \Delta t = 0
 \end{cases} 
\end{align}
$$

Fig.\ref{fig:STDP-window} uses $\tau_{\mathrm{LTP}}$ = 17 ms, $A_{\mathrm{LTP}}$ = 1.0, $\tau_{\mathrm{LTD}}$ = 34 ms, and $A_{\mathrm{LTD}}$ = - 0.5. These values, leading to a perfect compensation of LTP by LTD in terms of window area, are frequently used, though other values are possible. 
There are several ways to implement STDP in computational models distinguished by which pre- and post-synaptic spike pairs are allowed to interact to trigger changes in synaptic efficacies.  The two most often considered mechanisms are ``all-to-all,'' where every pre-synaptic spike interacts with every post-synaptic spike to trigger a weight change, and ``nearest-neighbour,'' where pre/post spikes only interact with the closest post/pre spikes. There are several possible variants of nearest-neihbor schemes reviewed by Morrison et al.\ \cite{morrison2008phenomenological}. Here, you should consider the ``narrow'' scheme depicted in Fig.~7 C of that publication. You can do this using two buffer variables that store the times of the last presynaptic and postsynaptic spikes.

Using the single LIF neuron model from unit 1.4 (without refractory period or SRA), leave out the inhibitory inputs for simplicity, or set their synaptic weight to zero. Reduce the number of excitatory inputs to only 2 and apply the STDP rule, as shown in the figure and the equations above, in these two synapses. Importantly, set $A_{\mathrm{LTP}}$ to 0.05 and $A_{\mathrm{LTD}}$ to -0.025, such that the change in weight is slow enough for us to see the evolution of the weights. Note also that if the initial weight in combination with the input firing rate and the number of inputs is too weak, there will be no post-spikes. Without post-spikes, STDP is not activated and the weight will remain unchanged, so in this model the synaptic drive must be strong from the start of the simulation to see any effect of STDP. Here, use an initial value of $w_\mathrm{e} = 1$ for each synapse, and apply two Poisson inputs with firing rate 5 Hz.
Plot the value of the weight changing over time. 

You should now see that the weights are growing in an unlimited fashion. As a result of the growing weight, the postsynaptic neuron will demonstrate an increasing firing rate. This problem is dealt with in later units, for now just put a maximum weight value $w_\mathrm{max}$ = 6, above which the weight cannot increase.
Uncontrolled growth of synaptic weights is a property of pure additive STDP models in excitatory synapses: since the pre-post spike combination is related to potentiation, STDP rewards causality in the case of an excitatory synapse. The increased excitatory weight then enhances the probability that the pre-spike causes a post-spike, and hence a positive feedback loop is created. Therefore, the weight will mostly undergo LTP and little LTD, despite the integral of the STDP window being 0. Possible mechanisms that the brain employs to limit this positive feedback have been frequently addressed in the last decade in modeling studies concerned with STDP.

Now change one of the Poisson inputs to a higher rate of 8 Hz. You now have two inputs with STDP, one with a higher and one with a lower firing rate (5 and 8 Hz). What do you see in the weights?  If it works well, the weight from the 8 Hz input should ``win the race''.

\begin{figure}[H]
\begin{center}
\includegraphics[width=10cm]{solution_figs/3_1_STDP_2synapses.png}
\caption{A LIF neuron with Poisson inputs and STDP in two excitatory input synapses with different firing rates (5 and 8 Hz). Left: The neuron is getting increasingly depolarised over time, resulting in more and more output spikes. Right: The cause of the increase in spiking is the growth in excitatory synaptic weights. Importantly, the synaptic weight of the strongly firing input (8 Hz) increases faster than that of the more weakly firing input (5 Hz).}
\end{center}
\end{figure}

## Correlated Spike Trains

When correlations between spike trains are nonzero, STDP can create additional competition between groups of synaptic inputs. Let us first see how we can implement such correlations in spike times. There are various ways to create correlations between spike trains, while maintaining Poisson-like statistics within the spike trains themselves. For example, here we are going to create Poisson spike trains that have instantaneous and exponential correlations with each other. We discuss one method, for creating such correlations, which has been described by Brette and colleagues \cite{Brette2008} (Method II). 

Consider a Poisson spike train $S_\mathrm{source}$ with firing rate $r_\mathrm{source}$. One way to create another spike train that has nonzero spike-time correlations with $S_\mathrm{source}$ is to simply copy spike times from $S_\mathrm{source}$ to the new spike train, which we call $S_\mathrm{new}$. If the spikes are copied with probability $p$, then the firing rate of any new spike train will be $ p\times r_\mathrm{source} $. More generally,

$$
r_\mathrm{new} = \sum^{M}_{k=1} p_k r_k,
$$
for $M$ different source spike trains. In this example, we just stick to one source spike train, from which we create multiple new spike trains.
If $p=1$, $S_\mathrm{new}$ is simply identical to $S_\mathrm{source}$. However according to equation 14, for any $p<1$, thinning of a Poisson spike train will result in $r_\mathrm{new} < r_\mathrm{source}$. What if we want the spike trains to have the same or higher firing rates? In order to compensate for the loss of spikes, the firing rate can be increased to reach $r_\mathrm{target}$ by adding additional spikes. This is done by superimposing another Poisson spike train $S_\mathrm{noise}$ onto $S_\mathrm{new}$, with firing rate $r_\mathrm{noise} = r_\mathrm{target}- p \times r_\mathrm{source}$. If this process is repeated independently for a number of $S_\mathrm{new}$  with the same $p$, $S_\mathrm{source}$, and independent $S_\mathrm{noise}$ for each spike train, one obtains a group of correlated spike trains that each fire with Poisson statistics and firing rate $r_\mathrm{target}$. Now, what are the correlations between such spike trains?

The cross-covariance between any two spike trains $S_i$ and $S_j$ is

$$
CCVF(s) = \langle S_i(t)S_j(t+s) \rangle - \langle S_i(t) \rangle \langle S_j(t) \rangle
$$

The angular brackets denote an average over time. This function is 0 for two independent Poisson spike trains, and is $r \delta (s)$ for two identical Poisson spike trains with firing rate $r$ (this is also called the autocovariance). 

The cross-covariance function between two new spike trains generated from the same source spike train by the thinning method described above, is

$$
CCVF(s) = p^2 r \delta(s) 
$$
with $r = r_\mathrm{source} = r_\mathrm{target}$. The area under the cross-covariance function, $p^2 r$ , normalised by the firing rate $r$, is the correlation $c$. 
We see here that $c$ = $p^2$. Therefore, if we choose $p$ = $\sqrt{c} $, the correlations between the spike trains will be $c$.
The method described above generates instantaneous correlations between spike trains. You can also make exponential cross-covariance between spike trains. In order to do so, you can shift the spikes randomly after copying them from $S_\mathrm{source}$. Randomly shifting the spikes does not change the properties of single spike trains: they remain Poisson and their firing rate does not change. 
Choose the random shift from an exponential probability density $P(x)$ for every spike. 
$$P(x) = \frac{1}{\tau_\mathrm{c}} e^{- \frac{x}{\tau_\mathrm{c}}}
\end{equation}
In this case, the cross-covariance function is a convolution of the probability density function:
\begin{equation}
CCVF(s) =   \int P(x)P(x+s)ds =  \frac{1}{2 \tau_\mathrm{c}} e^{- \frac{|s|}{\tau_\mathrm{c}}}
\end{equation}

We will refer to this type of correlation as exponential correlation. The variable $\tau_\mathrm{c}$ controls the width of the correlation.
First let us make groups of spike trains that have instantaneous correlations, without random spike time shifting. Create a group of 10 instantaneously correlated spike trains with $c$ = 0.1, and a group of 10 instantaneously correlated spike trains with $c$ = 0.1 and
Show the cross-correlogram (the cross-correlation histogram over different time lags) for each group; the correlation structure should be visible there. Within groups, you should see a large peak at zero lag, which should be absent when taking cross-correlations between groups.

Now apply exponential cross-correlations, by using $\tau_\mathrm{c} = $ 20 ms. How has the cross-correlogram changed? You should now see an exponential-like decay from both sides of the peak at lag zero.

In the next unit, we will apply these correlated spike trains to the neuron model with STDP.

\begin{figure}[H]
\begin{center}
\includegraphics[width=10cm]{solution_figs/3_2_cross_correlation.png}
\caption{An example of a cross-correlogram between two 10 Hz Poisson spike trains of length 15 seconds. The spike trains are correlated with c = 0.2, and spike correlation jitter $\tau_c = $ 20 ms. Spikes are binned in 5 ms bins. The vertical dotted line indicates the zero lag point. Although the spike correlation is jittered and not instantaneous, a peak can still be seen at zero lag.}
\end{center}
\end{figure}

\subsection{Correlations and STDP}

We have looked at STDP and at correlations in spike times separately, now let us put the two together. 
Create two separate groups with each 10 excitatory spike trains that have instantaneous correlation, and use $r_\mathrm{source}$  = $r_\mathrm{target}$ = 10 Hz, and $c$ = 0.1. Now input them onto the LIF neuron and put STDP in all the synapses. Set the STDP learning rate to be lower, $A_\mathrm{LTP}$ = 0.02 and $A_\mathrm{LTD}$  = -0.01, and start all excitatory weights at $w_\mathrm{e} =$ 0.5. Also include 10 inhibitory inputs with weight 1 and a firing rate of 10 Hz, without STDP.

What do you see in the weights of the excitatory input groups? Normally, the two groups should compete for the weight increase onto the neuron. Do both groups reach the maximum weight?
Now change the correlation between the spikes: group 1, $c_1$ = 0.1 and group 2, $c_2$ = 0.2, but maintain the same firing rate for all inputs. Simulate for long enough so you can see well what happens to the weights of the two groups. If everything is correct you should see that correlations as well as firing rates can influence the competition between groups of weights.
\\
\\

If you add exponential cross-covariance, but do not change $c1$ and $c2$, does it influence the weight evolution of the two competing groups? You may have to adjust the weights and learning rates to produce reasonable output firing rates.
Another effect you can look at is how the difference in initial weight in the two groups can influence weight competition by STDP. You can also test if an initial advantage in the weights for one group can overcome a weaker correlation or lower firing rate of that group in the competition for synaptic strength.  

%** show example of this?? **

\begin{figure}[H]
\begin{center}
\includegraphics[width=10cm]{solution_figs/3_3_STDP_correl.png}
\caption{A LIF neuron with two separately correlated excitatory input groups, one with c = 0.2 and one with c = 0.1. Left: The firing rate of the neuron increases over time. Time bins of 1 second are used to record the spikes. Right: Synaptic weights from two groups of excitatory inputs are shown evolving over time through STDP. The thin lines show individual weights, the thick line the average from the corresponding group.  Although the firing rates of the inputs groups are identical, the weights from the group with higher correlation increase faster on average.}
\end{center}
\end{figure}